# Building a comprehensive test suite for a simple function

## Aim: build a test suite for a "greatest common divisor function", `gcd`

> Definition: the greatest common divisor (GCD) of two or more integers, which are not all zero, is the largest positive integer that divides each of the integers. 
> For two integers $x$, $y$, the greatest common divisor of $x$ and $y$ is denoted
$\displaystyle \gcd(x, y)$.
>
> For example, $\displaystyle \gcd(8, 12) = 4$.
>
> ... gcd(0, 0) is commonly defined as 0 [*and this is our definition today*].
>
> _Source: Wikipedia, https://en.wikipedia.org/wiki/Greatest_common_divisor, retrieved on September 21 2023_


In [1]:
from gcd import gcd

## The "black-box" approach varies the inputs to a function and checks its outputs

<Image of a black box here>

The first approach we'll use is the "black-box" approach, which treats the function as a black box which we can't see inside. 

We'll look at inputs and check that they produce the correct outputs.

## Smoke test

Check for basic plausibility. Does it run without failing?

In [2]:
gcd(8, 12)  # runs without failing

4

Does it produce sensible results, like the correct datatype?

In [3]:
assert type(gcd(8, 12)) is int  

... or the correct sign (+ rather than -)?

In [4]:
assert gcd(8, 12) > 0

Does it produce the same result if a and b are swapped? (Only true for commutative operations)

In [5]:
assert gcd(12, 8)

## Essential tooling: `assert`

We can use assert statements in python to write tests.

The assert statement in python raises an Error if the result of a calculation is "Falsy":

In [6]:
assert True  # no Error

In [7]:
assert False  # raises an Exception

AssertionError: 

In [8]:
assert 1 == 1  # no Error

In [9]:
assert 1 > 0  # no Error

In [10]:
assert 0 > 1  # False, so raises an exception

AssertionError: 

In [11]:
assert "a"  # a string is truthy...

In [12]:
assert ""  # but an empty string is falsy

AssertionError: 

## Nominal case

Check for correct result in some nominal cases.

In [13]:
assert gcd(7, 21) == 7

In [14]:
assert gcd(20, 10) == 10

In [15]:
assert gcd(54, 24) == 6

## Boundary analysis

Check for correctness at the boundaries of the domain, or boundaries within parameters.

In [16]:
assert gcd(0, 17) == 0

In [17]:
assert gcd(1, 1_000_000) == 1

AssertionError: 

The `gcd` function operates on integers and has a boundary at zero

## Known special cases

Check special behavior at special values which might be explicitly defined.

In [18]:
assert gcd(0, 0) == 0

## Error guessing

Some input values cause more errors than others. 

You might be able to guess which errors will crop up.

### Zeros
Zeros often cause problems.

In [19]:
assert gcd(0, 100) == 0

### Values at the limit of a type's definition may cause issues

The "natural" maximum size of an integer might be $2^{63} - 1$ on a 64-bit system, so we'll check that.

> As of python 3, the only size limit for an integer is the size of memory [[1]](https://docs.python.org/3/library/sys.html#sys.maxsize). 

In [44]:
a = 2**63-1  # prime factors: 7, 73, 127, 337, 92737, 649657, https://www.wikidata.org/wiki/Q10571632
b = 649657 * 7 * 6  # the gcd is 649657 * 7 by construction
assert gcd(a, b) == 649657 * 7

### Mutable default values can cause very strange errors

In python, it's easy to introduce a fault which causes function to change its output each time you run it, even with the same inputs – check that a function returns the same output for the same input:

In [45]:
# Example of a function which displays this behavior
def append(value, the_list=[]):
    """
    Appends a value to a list, and if the list isn't given, return the value on a new list
    """
    the_list.append(value)
    return the_list

# Works fine if we give it a list to extend:
append(1, [])  # should return [1]

[1]

In [46]:
assert append(1, []) == [1]

If we don't give it a list to extend, it breaks:

In [47]:
assert append(1) == [1]

In [49]:
assert append(2) == [2]   # should return [2]!!!

AssertionError: 

What's going on? Let's try to debug this function:

In [51]:
append(2)

[1, 2, 2, 2]

The default value of `the_list` is getting extended each time we run the function.

You might think that you can check this by running a test like this:

In [52]:
assert append(3) == append(3)  # passes unexpectedly!

... but the error is so insidious that this test fails! Both functions are appending to the same list! 
You actually need to store a copy of the value from the first run and compare it later:

In [53]:
import copy


first_result = copy.deepcopy(append(4))
second_result = copy.deepcopy(append(4))

assert first_result == second_result, "%s != %s" % (first_result, second_result)

AssertionError: [1, 2, 2, 2, 3, 3, 4] != [1, 2, 2, 2, 3, 3, 4, 4]

In the context of our `gcd` function, this means:

In [55]:
first_gcd = copy.deepcopy(gcd(32, 8))
second_gcd = copy.deepcopy(gcd(32, 8))

assert first_gcd == second_gcd, "%s != %s" % (first_gcd, second_gcd)